# 05b - Vertex AI > Training > Custom Jobs - With Python Source Distribution

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05b`): An extension of `05a` that saves the script within a source distribution
This notebook trains the same Tensorflow Keras model from `05` by first modifying and saving the training code to a python script (same as `05a`).  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this notebook gives an example of making the shift.  

The source distribution is then used as an input for a Vertex AI > Training > Custom Job that is also assigned compute resources and a [pre-built container for custom training](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers) for executing the training in a managed service.  This is done with the [Vertex AI Python SDK](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#) using the class [`aiplatform.CustomJob()`](https://googleapis.dev/python/aiplatform/latest/aiplatform.html#google.cloud.aiplatform.CustomJob).

The training can be reviewed with Vertex AI's managed Tensorboard under Vertex AI > Experiments > Experiments, or by clicking on the `05b...` custom job under Vertex AI > Training > Custom Jobs and then clicking the 'Open Tensorboard' link.

<img src="architectures/overview/Training.png">

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Overview:
-  Setup
-  Connect to Tensorboard instance from 05
-  Create a `train.py` Python script that recreates the local training in 05
-  Build a Python source distribution that contains the `train.py` script
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Custom training job with aiplatform.CustomJob.from_local_script
      -  Run job with .run
   -  Upload Model to Vertex AI with aiplatform.Model.upload
   -  Create Endpoint with Vertex AI with aiplatform.Endpoint.create
      -  Deploy model to endpoint with .deploy 
-  Online Prediction demonstrated using Vertex AI Endpoint with deployed model
   -  Get records to score from BigQuery table
   -  Prediction with aiplatform.Endpoint.predict
   -  Prediction with REST
   -  Prediction with gcloud (CLI)

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)



-----
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05b_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05b_console.png">

---
## Setup

inputs:

In [5]:
REGION = 'us-central1'
PROJECT_ID='ma-mx-presales-lab'
DATANAME = 'fraud'
NOTEBOOK = '05b'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [6]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [8]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = "vertex-ai-mlops-bucket"
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [9]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'825075454589-compute@developer.gserviceaccount.com'

environment:

In [10]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [11]:
#tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
#if tb:
#    tb = tb[0]
#else:
#    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

#tb.resource_name

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [12]:
!mkdir -p {DIR}/source/trainer

In [31]:
%%writefile {DIR}/source/trainer/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
args = parser.parse_args()

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# Make a list of columns to omit
OMIT = args.var_omit + ['splits']

# use schema to prepare a list of columns to read from BigQuery
selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

# all the columns in this data source are either float64 or int64
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in schema[~schema.column_name.isin(OMIT)].data_type.tolist()]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().prefetch(1).map(transTable).shuffle(args.batch_size*10).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().prefetch(1).map(transTable).batch(args.batch_size)

# Logistic Regression

# model input definitions
feature_columns = {header: tf.feature_column.numeric_column(header) for header in selected_fields if header != args.var_target}
feature_layer_inputs = {header: tf.keras.layers.Input(shape = (1,), name = header) for header in selected_fields if header != args.var_target}

# feature columns to a Dense Feature Layer
feature_layer_outputs = tf.keras.layers.DenseFeatures(feature_columns.values())(feature_layer_inputs)

# batch normalization then Dense with softmax activation to nclasses
layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

# the model
model = tf.keras.Model(
    inputs = feature_layer_inputs,
    outputs = layers
)
opt = tf.keras.optimizers.SGD() #SGD or Adam
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)

# setup tensorboard logs and train
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR'], histogram_freq=1)


# Checkpoint callback
#print(os.getenv("AIP_CHECKPOINT_DIR"))
#print
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=os.environ['AIP_CHECKPOINT_DIR'], save_weights_only=True, monitor='val_loss',
                                                   mode='min',  # 'max'
                                                   save_best_only=True, verbose=1)


history = model.fit(train, epochs = args.epochs, callbacks = [[checkpoint_cb, tensorboard_callback]], validation_data = validate)

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))

Overwriting temp/05b/source/trainer/train.py


### Assemble Python Source Distribution

create `setup.py` file:

In [32]:
%%writefile {DIR}/source/setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['tensorflow_io']

setup(
    name = 'trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)

Overwriting temp/05b/source/setup.py


add `__init__.py` file to the trainer modules folder:

In [33]:
!touch {DIR}/source/trainer/__init__.py

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [34]:
%cd {DIR}

!rm -f source.tar source.tar.gz
!tar cvf source.tar source
!gzip source.tar
!gsutil cp source.tar.gz {URI}/{TIMESTAMP}/source.tar.gz

temp = '../'*(DIR.count('/')+1) 
%cd {temp}

/home/jupyter/GCP_Vertex_AI/Vertex-ai-mlops/temp/05b
source/
source/setup.py
source/trainer/
source/trainer/__init__.py
source/trainer/train.py
Copying file://source.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      
/home/jupyter/GCP_Vertex_AI/Vertex-ai-mlops


### Setup Training Job

In [35]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK
]

In [36]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [f"{URI}/{TIMESTAMP}/source.tar.gz"],
            "python_module": "trainer.train",
            "args": CMDARGS
        }
    }
]

In [37]:
customJob = aiplatform.CustomJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [38]:
customJob.run(
    service_account = SERVICE_ACCOUNT,
    #tensorboard = tb.resource_name
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/825075454589/locations/us-central1/customJobs/6262304210126110720
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/825075454589/locations/us-central1/customJobs/6262304210126110720')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6262304210126110720?project=825075454589
INFO:google.cloud.aiplatform.jobs:CustomJob projects/825075454589/locations/us-central1/customJobs/6262304210126110720 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/825075454589/locations/us-central1/customJobs/6262304210126110720 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/825075454589/locations/us-central1/

In [39]:
customJob.display_name

'05b_fraud_20220405225214'

---
## Serving

### Upload The Model

In [40]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = f"{URI}/{TIMESTAMP}/model",
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/825075454589/locations/us-central1/models/1966828389998264320/operations/4114120241938694144
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/825075454589/locations/us-central1/models/1966828389998264320
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/825075454589/locations/us-central1/models/1966828389998264320')


In [41]:
model.display_name

'05b_fraud_20220405225214'

### Create An Endpoint

In [42]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/825075454589/locations/us-central1/endpoints/4871883246117322752/operations/2907155541803401216
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/825075454589/locations/us-central1/endpoints/4871883246117322752
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/825075454589/locations/us-central1/endpoints/4871883246117322752')


In [43]:
endpoint.display_name

'05b_fraud_20220405225214'

### Deploy Model To Endpoint

In [44]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/825075454589/locations/us-central1/models/1966828389998264320 to Endpoint : projects/825075454589/locations/us-central1/endpoints/4871883246117322752
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/825075454589/locations/us-central1/endpoints/4871883246117322752/operations/3539911289448955904
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/825075454589/locations/us-central1/endpoints/4871883246117322752


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [45]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [46]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,6126,-0.130702,1.026568,2.692793,2.659141,0.764928,1.800330,0.317986,-0.392976,0.851460,...,-0.370958,-1.058635,-0.498915,0.054906,-0.590204,-0.537678,0.0,0,70526fff-bdf3-459f-bbc6-a18e8133bb35,TEST
1,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e8b3c609-62a9-4ec3-9ff8-fd0974b07602,TEST
2,35599,1.168909,-0.139981,-0.095518,1.735426,1.699903,4.646212,-1.191502,1.205700,0.333882,...,-0.163863,1.045593,0.775343,0.228745,0.050846,0.020019,0.0,0,c5ac2c63-c26b-492e-bc8e-5f98e92ec866,TEST
3,49561,1.333331,-0.845997,1.161578,-0.610965,-1.635783,-0.198304,-1.331531,0.212857,-0.208834,...,-0.095332,0.367347,0.398274,-0.059810,0.041674,0.011653,0.0,0,aca44b9b-e816-4767-b636-0356637c286b,TEST


In [47]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [48]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [49]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999978065, 2.19029662e-05]], deployed_model_id='9091703320405344256', explanations=None)

In [50]:
prediction.predictions[0]

[0.999978065, 2.19029662e-05]

In [51]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [52]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [53]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999978065,
      2.19029662e-05
    ]
  ],
  "deployedModelId": "9091703320405344256",
  "model": "projects/825075454589/locations/us-central1/models/1966828389998264320",
  "modelDisplayName": "05b_fraud_20220405225214"
}


### Get Predictions: gcloud (CLI)

In [54]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999978065, 2.19029662e-05]]


---
## Remove Resources
see notebook "99 - Cleanup"